In [4]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from math import sqrt
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.callbacks import EarlyStopping
from gensim.models import Word2Vec
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
code = pd.read_excel(r"/content/NNDL.xlsx")
code

,Question,Correct_Code,Code_with_Error,Type_of_Error,Number_of_Loops,Header_and_Main_Declaration,Incomprehensible_Code,Comprehensible_Code_with_logical_errors,Comprehensible_code_with_syntax_errors,Correct_code_and_output,Total_marks
0,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() \n\ndouble n1...,"1c,1e",0,2,2.0,2,0,0,6
1,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,"#include <stdio.h>\n\nint main\n\ndouble n1, n...","1c,1e,1d",0,0,2.0,2,0,0,4
2,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,"1b,1e",0,2,2.0,0,1,0,5
3,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,1e,0,2,2.0,2,1,0,7
4,Largest among three numbers,#include <stdio.h>\n\nint main() {\n\ndouble n...,#include <stdio.h>\n\nint main() {\n\ndouble n...,2b,0,2,2.0,0,2,0,6
...,...,...,...,...,...,...,...,...,...,...,...
114,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint main(...,2a,0,2,2.0,1,2,2,9
115,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint mian(...,"1a,1c",0,2,2.0,2,0,0,6
116,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint main(...,3a,0,2,2.0,2,2,2,10
117,Calculate the roots of an equation,#include<stdio.h>\n#include<math.h>\nint main(...,#include<stdio.h>\n#include<math.h>\nint min()...,"1a,1e,2a",0,1,1.0,1,0,0,3


In [6]:
# Merge columns
merged_data = code['Question'] + ' ' + code['Code_with_Error']
merged_data

0      Largest among three numbers #include <stdio.h>...
1      Largest among three numbers #include <stdio.h>...
2      Largest among three numbers #include <stdio.h>...
3      Largest among three numbers #include <stdio.h>...
4      Largest among three numbers #include <stdio.h>...
                             ...                        
114    Calculate the roots of an equation #include<st...
115    Calculate the roots of an equation #include<st...
116    Calculate the roots of an equation #include<st...
117    Calculate the roots of an equation #include<st...
118    Calculate the roots of an equation #include<st...
Length: 119, dtype: object

In [7]:
# Remove rows with missing values
merged_data = merged_data.dropna()

In [8]:
# Drop rows with missing values
df_clean = code.dropna(subset=['Total_marks'])

In [9]:
# Tokenize the merged data
tokenized_data = [sentence.split() for sentence in merged_data]

In [10]:
# Train Word2Vec model
w2v_model = Word2Vec(tokenized_data, vector_size=100, window=5, min_count=1, workers=4)

In [11]:
# Convert tokens to vectors
vectorized_data = []
total_marks = []
for sentence, marks in zip(tokenized_data, df_clean['Total_marks']):
    vectors = []
    for token in sentence:
        if token in w2v_model.wv:
            vectors.append(w2v_model.wv[token])
        else:
            vectors.append(np.zeros(100))
    vectorized_data.append(vectors)
    total_marks.append(marks)

In [12]:
vectorized_data = np.array(vectorized_data)
total_marks = np.array(total_marks)

<ipython-input-12-7480c13aae3a>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  vectorized_data = np.array(vectorized_data)


In [13]:
# Split the dataset
X_train, X_test, y_train, y_test = train_test_split(vectorized_data, total_marks, test_size=0.25, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=42)

In [14]:
# Pad sequences to a fixed length
max_sequence_length = 122
X_train_pad = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test_pad = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')
X_val_pad = pad_sequences(X_val, maxlen=max_sequence_length, padding='post')

In [15]:
# Define the LSTM model
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(max_sequence_length, X_train_pad.shape[2])),
    tf.keras.layers.LSTM(units=64),
    tf.keras.layers.Dense(1)
])

In [16]:
# Compile the model
model.compile(loss='mse', optimizer='adam')

In [36]:

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5)

# Train the model
model.fit(X_train_pad, y_train, validation_data=(X_val_pad, y_val), epochs=50, batch_size=64, callbacks=[early_stopping])

Epoch 1/50
2/2 [==============================] - 0s 126ms/step - loss: 5.5653 - val_loss: 7.9422
Epoch 2/50
2/2 [==============================] - 0s 87ms/step - loss: 5.6550 - val_loss: 8.0088
Epoch 3/50
2/2 [==============================] - 0s 99ms/step - loss: 5.6972 - val_loss: 8.0134
Epoch 4/50
2/2 [==============================] - 0s 110ms/step - loss: 5.7002 - val_loss: 7.9828
Epoch 5/50
2/2 [==============================] - 0s 86ms/step - loss: 5.6794 - val_loss: 7.9290
Epoch 6/50
2/2 [==============================] - 0s 95ms/step - loss: 5.6411 - val_loss: 7.8949
Epoch 7/50
2/2 [==============================] - 0s 82ms/step - loss: 5.6157 - val_loss: 7.8437
Epoch 8/50
2/2 [==============================] - 0s 87ms/step - loss: 5.5770 - val_loss: 7.7355
Epoch 9/50
2/2 [==============================] - 0s 90ms/step - loss: 5.5028 - val_loss: 7.5973
Epoch 10/50
2/2 [==============================] - 0s 83ms/step - loss: 5.4423 - val_loss: 7.5079
Epoch 11/50
2/2 [==========

In [37]:
# Convert sequences to a compatible format
X_train_pred = pad_sequences(X_train, maxlen=max_sequence_length, padding='post')
X_test_pred = pad_sequences(X_test, maxlen=max_sequence_length, padding='post')

# Make predictions
y_train_pred = model.predict(X_train_pred)
y_test_pred = model.predict(X_test_pred)

1/1 [==============================] - 0s 42ms/step


In [38]:

# Calculate evaluation metrics
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mape = np.mean(np.abs((y_train - y_train_pred) / y_train)) * 100
train_r2 = r2_score(y_train, y_train_pred)

test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mape = np.mean(np.abs((y_test - y_test_pred) / y_test)) * 100
test_r2 = r2_score(y_test, y_test_pred)


In [39]:
# Print the evaluation metrics
print(f"Train RMSE: {train_rmse:.2f}")
print(f"Train MAE: {train_mae:.2f}")
print(f"Train MAPE: {train_mape:.2f}")
print(f"Train R2: {train_r2:.2f}")

print(f"Test RMSE: {test_rmse:.2f}")
print(f"Test MAE: {test_mae:.2f}")
print(f"Test MAPE: {test_mape:.2f}")
print(f"Test R2: {test_r2:.2f}")

Train RMSE: 2.33
Train MAE: 1.92
Train MAPE: 44.54
Train R2: -0.00
Test RMSE: 2.36
Test MAE: 2.02
Test MAPE: 42.77
Test R2: -0.04
